In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

from features.data_provider import get_feature_columns, get_whole_dataset, set_feature_columns
from simulation.predictor import MaxProbabilityOutcomePredictor
from models.outcome_model import get_model
from simulation.analyse import get_win_probabilities, get_simulations
from simulation.simulation import run_actual_tournament_simulation
from db.simulation_table import get_simulation_results, delete_all
from bet.unit_strategy import UnitStrategy
from bet.kelly_strategy import KellyStrategy
from notebook_helpers import plot_bank_and_bets, write_report

In [2]:
all_features = get_feature_columns()
player_features = ['rating_diff', 'potential_diff', 'height_diff','weight_diff','age_diff',
                   'weak_foot_diff','internationl_repuatiotion_diff','crossing_diff','finishing_diff',
                   'heading_accuracy_diff','short_passing_diff','dribbling_diff','fk_accuracy_diff',
                   'long_passing_diff','ball_control_diff','acceleration_diff','sprint_speed_diff',
                   'reactions_diff','shot_power_diff','stamina_diff','strength_diff','long_shots_diff',
                   'aggression_diff','penalties_diff','marking_diff','standing_tackle_diff',
                  'gk_diving_diff', 'gk_handling_diff', 'gk_kicking_diff', 'gk_reflexes_diff']

other_features = ['elo_diff', 'away_goal_mean', 'away_goals_with_home', 
                  'goal_diff_with_away', 'home_goal_mean', 'home_goals_with_away']

assert (len(player_features) + len(other_features)) == len(all_features)

In [3]:
def simulate_betting_strategies(features, match_template_file, 
                                bet_file, filter_start=None, filter_end=None, interval=None,
                                iter_n=10):
    unit_profit = []
    kelly_profit = []
    accuracies = []

    set_feature_columns(features)
    
    X, y = get_whole_dataset("home_win", filter_start=filter_start, filter_end=filter_end, interval=interval)

    print(X.shape)
    for i in range(10):
        model = get_model(X=X, y=y, n_estimators=2000)
        predictor = MaxProbabilityOutcomePredictor(model)
        match_template = pd.read_csv(match_template_file)
        run_actual_tournament_simulation(match_template, predictor)
        tournament_simulation = get_simulation_results()
        tournament_simulation["true_outcome"] = np.sign(tournament_simulation["home_score"] - tournament_simulation["away_score"])
        delete_all()

        accuracy = sum(tournament_simulation["outcome"] == tournament_simulation["true_outcome"]) / tournament_simulation.shape[0]
        accuracies.append(accuracy)

        match_bets = pd.read_csv(bet_file)

        odds = match_bets[["1", "X", "2"]].values
        probabilities = tournament_simulation[["home_win_prob", "draw_prob", "away_win_prob"]].values        
        y_pred = tournament_simulation["outcome"].values
        y_true = tournament_simulation["true_outcome"].values
        
        unit_strategy = UnitStrategy(y_pred, y_true)
        unit_strategy.run(odds)

        kelly_strategy = KellyStrategy(y_true)
        kelly_strategy.run(odds, probabilities)

        unit_profit.append(unit_strategy.get_total_profit())
        kelly_profit.append(kelly_strategy.get_total_profit())

    return accuracies, unit_profit, kelly_profit

In [ ]:
filename = "outcome_report.txt"

In [4]:
#### WC 2018
match_template = 'data/original/wc_2018_games_real.csv'
betting_file = 'data/original/wc_2018_bets.csv'

In [5]:
acc, unit, kelly = simulate_betting_strategies(all_features, match_template, betting_file)
write_report(acc, unit, kelly, "WC2018_allfeatures", filename)

(4739, 36)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid

AVG Accuracy:  0.578125 0.0
AVG Unit profit:  1.0765625 0.0
AVG Kelly profit:  1.12131402225 0.0557941385671


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))


In [6]:
acc, unit, kelly = simulate_betting_strategies(other_features, match_template, betting_file)
write_report(acc, unit, kelly, "WC2018_otherfeatures", filename)

(4739, 6)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid

AVG Accuracy:  0.5296875 0.00841432001115
AVG Unit profit:  0.993328125 0.0210810204646
AVG Kelly profit:  0.59275882997 0.0302914684735


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))


In [7]:
acc, unit, kelly = simulate_betting_strategies(player_features, match_template, betting_file)
write_report(acc, unit, kelly, "WC2018_playerfeatures", filename)

(4739, 30)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: divide 

AVG Accuracy:  0.609375 0.0
AVG Unit profit:  1.19953125 0.0
AVG Kelly profit:  1.62338820854 0.0785492622399


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))


In [8]:
#### WC 2014
match_template = 'data/original/wc_2014_games_real.csv'
betting_file = 'data/original/wc_2014_bets.csv'

In [9]:
acc, unit, kelly = simulate_betting_strategies(all_features, match_template, betting_file,
                                              filter_start="2014-06-12")
write_report(acc, unit, kelly, "WC2014_allfeatures", filename)

(2917, 36)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid

AVG Accuracy:  0.59375 0.0
AVG Unit profit:  1.1040625 2.22044604925e-16
AVG Kelly profit:  1.26515408594 0.0569377290867


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))


In [10]:
acc, unit, kelly = simulate_betting_strategies(other_features, match_template, betting_file,
                                               filter_start="2014-06-12")
write_report(acc, unit, kelly, "WC2014_otherfeatures", filename)

(2917, 6)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid

AVG Accuracy:  0.5890625 0.010004881621
AVG Unit profit:  1.110578125 0.0310073430378
AVG Kelly profit:  1.37848708322 0.0736357224299


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))


In [11]:
acc, unit, kelly = simulate_betting_strategies(player_features, match_template, betting_file,
                                               filter_start="2014-06-12")
write_report(acc, unit, kelly, "WC2014_playerfeatures", filename)

(2917, 30)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid

AVG Accuracy:  0.6015625 0.0078125
AVG Unit profit:  1.13140625 0.0243955242117
AVG Kelly profit:  1.13984331337 0.0148545428905


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))


In [12]:
#### WC 2010
match_template = 'data/original/wc_2010_games_real.csv'
betting_file = 'data/original/wc_2010_bets.csv'

In [13]:
acc, unit, kelly = simulate_betting_strategies(all_features, match_template, betting_file,
                                              filter_start="2010-06-11")
write_report(acc, unit, kelly, "WC2010_allfeatures", filename)

(1198, 36)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid

AVG Accuracy:  0.5625 0.0
AVG Unit profit:  1.06828125 0.0
AVG Kelly profit:  1.08271545305 0.0290537809537


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))


In [14]:
acc, unit, kelly = simulate_betting_strategies(other_features, match_template, betting_file,
                                               filter_start="2010-06-11")
write_report(acc, unit, kelly, "WC2010_otherfeatures", filename)

(1198, 6)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: divide by zero encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: divide by zero encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: inval

AVG Accuracy:  0.565625 0.00625
AVG Unit profit:  1.0904375 0.019625
AVG Kelly profit:  1.66718299752 0.160572481687


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))


In [15]:
acc, unit, kelly = simulate_betting_strategies(player_features, match_template, betting_file,
                                               filter_start="2010-06-11")
write_report(acc, unit, kelly, "WC2010_playerfeatures", filename)

(1198, 30)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: divide by zero encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: divide by zero encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: inval

AVG Accuracy:  0.5390625 0.0078125
AVG Unit profit:  1.015734375 0.0363305980341
AVG Kelly profit:  1.05576207044 0.193651934595


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: divide by zero encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))


In [16]:
from sklearn.model_selection import train_test_split

for features_group in [all_features, other_features, player_features]:
    set_feature_columns(features_group)
    X, y = get_whole_dataset("home_win")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
    model = get_model(X=X_train, y=y_train, n_estimators=500)

    score = model.score(X_test, y_test)
    print("Score: ", score)

Score:  0.572784810127
Score:  0.582805907173
Score:  0.561708860759


In [60]:
#### WC 2018
match_template = 'data/original/wc_2018_games_real.csv'
betting_file = 'data/original/wc_2018_bets.csv'

for idx, interval in enumerate([("2004-06-12", "2010-06-11"), ("2010-06-11", "2014-06-12"), ("2014-06-12", "2018-06-14")]):
    acc, unit, kelly = simulate_betting_strategies(all_features, match_template, betting_file,
                                               interval=interval, iter_n=10)
    write_report(acc, unit, kelly, f"WC2018_allfeatures_interval_{idx}", filename)

(1198, 36)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid

AVG Accuracy:  0.5828125 0.00716027452337
AVG Unit profit:  1.105859375 0.0175426725823
AVG Kelly profit:  0.745008724654 0.0355730740116
(1719, 36)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid

AVG Accuracy:  0.58125 0.00625
AVG Unit profit:  1.08728125 0.0214375
AVG Kelly profit:  1.04280718764 0.0538726424472
(1822, 36)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid

AVG Accuracy:  0.559375 0.0136215591986
AVG Unit profit:  1.03134375 0.0405805766731
AVG Kelly profit:  0.84621130866 0.0204503726618


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))


In [17]:
#### WC 2018
match_template = 'data/original/wc_2018_games_real.csv'
betting_file = 'data/original/wc_2018_bets.csv'

for idx, interval in enumerate([("2004-06-12", "2010-06-11"), ("2010-06-11", "2014-06-12"), ("2014-06-12", "2018-06-14")]):
    acc, unit, kelly = simulate_betting_strategies(other_features, match_template, betting_file,
                                               interval=interval, iter_n=10)
    write_report(acc, unit, kelly, f"WC2010_otherfeatures_interval_{idx}", filename)

(1198, 6)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: divide by zero encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: divide by zero encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: inval

AVG Accuracy:  0.521875 0.0076546554462
AVG Unit profit:  0.970234375 0.0154574216726
AVG Kelly profit:  0.439433056776 0.0696125639605
(1719, 6)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid

AVG Accuracy:  0.534375 0.00625
AVG Unit profit:  0.9851875 0.0138125
AVG Kelly profit:  0.722931418098 0.0327743466517
(1822, 6)


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: divide by zero encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: divide by zero encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: inval

AVG Accuracy:  0.5109375 0.010004881621
AVG Unit profit:  0.931484375 0.0308411734296
AVG Kelly profit:  0.610544806287 0.171281834297


/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: invalid value encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))
/Users/villetoiviainen/Documents/dev/world_cup/bet/kelly_strategy.py:12: RuntimeWarning: divide by zero encountered in log
  return -(p1 * np.log(1 + o1*a - b - c) + p2 * np.log(1 + o2*b - a - c) + p3*np.log(1 + o3*c - a - b))


In [57]:
def generate_table(string):
    string = string.strip()
    profit, std = string.split()
    profit = (float(profit) - 1) * 100
    profit = np.around(profit, 6)
    std = float(std) * 100
    std = np.around(std, 4)
    s = f"{profit}?% $?pm$ {std}"
    s = s.replace("?", '\\')
    print(s)

In [59]:
generate_table("1.2520893706610998 0.038637586924018294")#

25.208937\% $\pm$ 3.8638
